In [1]:
%matplotlib qt5
#%% Import modules
from mats_utils.rawdata.read_data import read_MATS_data
import datetime as DT
from mats_utils.plotting.plotCCD import *
from math import *
from mats_l1_processing.pointing import pix_deg
from mats_utils.geolocation.coordinates import NADIR_geolocation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R
from scipy.optimize import minimize_scalar
from skyfield.positionlib import ICRF
from skyfield.api import wgs84 
from skyfield.units import Distance
from skyfield import api as sfapi
from scipy.interpolate import RegularGridInterpolator
from tqdm import tqdm

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""


In [2]:
#%% local variables
# value of a saturated pixel 
sat_val = 32880

# times for start and stop
start_time = DT.datetime(2023, 1, 12, 3, 30, 0)
stop_time = DT.datetime(2023, 1, 12, 4, 0, 0)
start_time = DT.datetime(2023, 3, 13, 3, 27, 0)
stop_time = DT.datetime(2023, 3, 13, 3, 29, 0)
start_time = DT.datetime(2023, 3, 20, 12, 0, 0)
stop_time = DT.datetime(2023, 3, 20, 13, 0, 0)
start_time = DT.datetime(2022, 12, 20, 20, 30, 0)
stop_time = DT.datetime(2022, 12, 20, 22, 0, 0)
# filter selecting Nadir chanel
filter={'CCDSEL': [7,7]}

In [3]:
#%% reading mesurements
df1a= read_MATS_data(start_time, stop_time,filter,level='1a',version='0.5')
print(len(df1a))

# displaying keys
pd.set_option('display.max_rows', 100)
df1a.dtypes
df1a_tot = df1a
df1a = df1a_tot[-100:-1]

1065


In [4]:
df1a = df1a_tot[df1a_tot['nadir_sza']>105.7]
df1a = df1a_tot[df1a_tot['satlat']>82]
print(len(df1a))
for i in range(len(df1a_tot)):
    print(df1a_tot.iloc[i]['satlat'])

37
31.761392201629793
31.88416931617772
32.00694511695827
32.12965736196163
32.252485222847106
32.37525571837931
32.49802106254236
32.620782441797004
32.74348207239698
32.86629498097175
32.98904435170434
33.11179415391154
33.234539077719774
33.35727942145472
33.48001888166203
33.60275539605651
33.72548785313979
33.84821599616887
33.97093825266177
34.093657521366666
34.21636920750708
34.339079193010306
34.4617860145289
34.584491912296095
34.70719153364755
34.82988998164027
34.95258468983052
35.07527599451253
35.19796434505041
35.32064638441866
35.44331883453163
35.56598792027082
35.68865572636316
35.811319687134905
35.93398638190079
36.05665109790336
36.179310669423145
36.30196151208898
36.42460341661966
36.547237986503056
36.66987330651004
36.792507003372364
36.91513619785901
37.03775988606889
37.160373761258604
37.282979784589564
37.40558114499309
37.52818200293927
37.650775378399835
37.77336366039464
37.89595137186424
38.018539658497495
38.14112015197445
38.26368967411805
38.38624611

In [8]:
#%% geolocating images
from tqdm import tqdm

TX,TY = np.meshgrid(range(45),range(5))

n = len(df1a)
a,b = np.shape(df1a.iloc[0]['IMAGE'])
lat_points = np.zeros((n,a,b))
lon_points = np.zeros((n,a,b))
sza_points = np.zeros((n,a,b))
im_points = np.zeros((n,a,b))


for i in tqdm(range(n)):
    ccditem = df1a.iloc[i]
    im = ccditem['IMAGE']
    lat_map,lon_map,sza_map = NADIR_geolocation(ccditem,x_sample=None,y_sample=None,interp_method='cubic')
    lat_points[i,:,:] = lat_map
    lon_points[i,:,:] = lon_map    
    sza_points[i,:,:] = sza_map
    im_points[i,:,:] = im
    #print(f"image n# {i}/{n}")

 28%|██▊       | 28/99 [02:56<07:26,  6.29s/it] 


KeyboardInterrupt: 

In [11]:
from scipy.interpolate import griddata


# latmin,latmax = -31.5,-30
# lonmin,lonmax = 28.1,29.7
# nb_lat = 30
# nb_lon = 30

# lo = np.linspace(lonmin,lonmax,nb_lon)
# la = np.linspace(latmin,latmax,nb_lat)
# new_lon,new_lat = np.meshgrid(lo,la)


latmin,latmax = np.min(lat_points),np.max(lat_points)
lonmin,lonmax = np.min(lon_points),np.max(lon_points)
nb_lat = ceil((latmax-latmin)/0.05)
nb_lon = ceil((lonmax-lonmin)/0.05)

lo = np.linspace(lonmin,lonmax,nb_lon)
la = np.linspace(latmin,latmax,nb_lat)
new_lon,new_lat = np.meshgrid(lo,la)


def simple_resampler(data,lat,lon,new_lat,new_lon,method='nearest'):
    points = np.zeros((np.shape(lat.ravel())[0],2))
    points[:,0] = lon.ravel()
    points[:,1] = lat.ravel()
    new_data = griddata(points, data.ravel(), (new_lon, new_lat), method='nearest')
    return new_data

def average_stacking(data,lat,lon,new_lat,new_lon):
    lat_step = new_lat[1]-new_lat[0]
    lon_step = new_lon[1]-new_lon[0]
    lon_bins = np.linspace(new_lon[0]-lon_step*0.5,new_lon[-1]+lon_step*0.5,len(new_lon))
    lat_bins = np.linspace(new_lat[0]-lat_step*0.5,new_lat[-1]+lat_step*0.5,len(new_lat))
    data_points = data.ravel()
    lon_ind = np.digitize(lon.ravel(),lon_bins)
    lat_ind = np.digitize(lat.ravel(),lat_bins)
    new_im = np.zeros((len(lat_bins),len(lon_bins)))
    new_im_nb = np.zeros((len(lat_bins),len(lon_bins)))
    for i in tqdm(range(len(lon.ravel()))):
        new_im[lat_ind[i]-1,lon_ind[i]-1] += data_points[i]
        new_im_nb[lat_ind[i]-1,lon_ind[i]-1] += 1    
    for i in tqdm(range(len(lat_bins))):
        for j in range(len(lon_bins)):
            if new_im_nb[i,j] > 0:
                new_im[i,j] = new_im[i,j]/new_im_nb[i,j]
            else :
                new_im[i,j] = None

    # data_crs = ccrs.PlateCarree()
    # plt.figure('average nb')
    # #projection = ccrs.Robinson()
    # projection = ccrs.PlateCarree()
    # #projection = ccrs.Orthographic(central_latitude=-90,central_longitude=0)
    # ax = plt.axes(projection=projection)
    # ax.set_global()
    # ax.coastlines()
    # ax.gridlines()
    # c= ax.pcolor(new_lon,new_lat,new_im_nb, transform=data_crs)
    # plt.colorbar(c,ax=ax)
    # plt.show()    
    return (new_im[:-1,:-1])










In [12]:
latmin,latmax = np.min(lat_points),np.max(lat_points)
lonmin,lonmax = np.min(lon_points),np.max(lon_points)
nb_lat = ceil((latmax-latmin)/0.05)
nb_lon = ceil((lonmax-lonmin)/0.05)

lo = np.linspace(lonmin,lonmax,nb_lon)
la = np.linspace(latmin,latmax,nb_lat)
new_lon,new_lat = np.meshgrid(lo,la)


new_im3 = average_stacking(im_points,lat_points,lon_points,la,lo)

IndexError: index 1 is out of bounds for axis 0 with size 0

In [ ]:
data_crs = ccrs.PlateCarree()
plt.close('average')
plt.figure('average')
#projection = ccrs.Robinson()
projection = ccrs.PlateCarree()
ax = plt.axes(projection=projection)
ax.set_global()
ax.coastlines()
ax.gridlines()
c = ax.pcolor(new_lon,new_lat,new_im3, transform=data_crs)
plt.colorbar(c,ax=ax)
plt.show()




In [ ]:
# polar projection
data_crs = ccrs.PlateCarree()
plt.close('average polar')
plt.figure('average polar')
projection = ccrs.Orthographic(central_latitude=80,central_longitude=-170)
ax = plt.axes(projection=projection)
ax.set_global()
ax.coastlines()
ax.gridlines()
c = ax.pcolor(new_lon,new_lat,new_im3, transform=data_crs)
plt.colorbar(c,ax=ax)
plt.show()

plt.close('original polar')
plt.figure('original polar')
ax = plt.axes(projection=projection)
ax.set_global()
ax.coastlines()
ax.gridlines()
for i in tqdm(range(0,n,5)):
    c = ax.pcolor(lon_points[i,:,:],lat_points[i,:,:],im_points[i,:,:], transform=data_crs)
plt.colorbar(c,ax=ax)
plt.show() 




In [ ]:
#new_im = simple_resampler(im_points,lat_points,lon_points,new_lat,new_lon)
#new_im2 = simple_resampler(im_points,lat_points,lon_points,new_lat,new_lon,method='linear')

data_crs = ccrs.PlateCarree()
projection = ccrs.PlateCarree()

plt.figure('original')
ax = plt.axes(projection=projection)
ax.set_global()
ax.coastlines()
ax.gridlines()
for i in tqdm(range(0,n,5)):
    ax.pcolor(lon_points[i,:,:],lat_points[i,:,:],im_points[i,:,:], transform=data_crs)
plt.show() 


plt.figure('nearest')
ax = plt.axes(projection=projection)
ax.set_global()
ax.coastlines()
ax.gridlines()
ax.pcolor(new_lon,new_lat,new_im, transform=data_crs)
plt.show() 


plt.figure('linear')
ax = plt.axes(projection=projection)
ax.set_global()
ax.coastlines()
ax.gridlines()
ax.pcolor(new_lon,new_lat,new_im2, transform=data_crs)
plt.show()



In [ ]:


data_crs = ccrs.PlateCarree()
projection = ccrs.PlateCarree()
#projection = ccrs.Robinson()
#projection = ccrs.Mollweide()
projection = ccrs.Orthographic(central_latitude=80,central_longitude=-170)
plt.close('mosaic')
plt.figure('mosaic')
ax = plt.axes(projection=projection)
ax.set_global()
ax.coastlines()
ax.gridlines()
for i in tqdm(range(0,n,5)):
    c = ax.pcolor(lon_points[i,:,:],lat_points[i,:,:],im_points[i,:,:], transform=data_crs)
plt.colorbar(c,ax=ax)
#plt.pcolormesh(np.reshape(lon_points[0:100,:,:],(a,b*100)), np.reshape(lat_points[0:100,:,:],(a,b*100)), np.reshape(im_points[0:100,:,:],(a,b*100)))

plt.show()  

plt.close('newest')
plt.figure('newest')
ax = plt.axes(projection=projection)
ax.set_global()
ax.coastlines()
ax.gridlines()
for i in tqdm(range(0,n,1)):
    c = ax.pcolor(lon_points[i,:,:],lat_points[i,:,:],im_points[i,:,:], transform=data_crs)
plt.colorbar(c,ax=ax)
#plt.pcolormesh(np.reshape(lon_points[0:100,:,:],(a,b*100)), np.reshape(lat_points[0:100,:,:],(a,b*100)), np.reshape(im_points[0:100,:,:],(a,b*100)))

plt.show()   

plt.close('oldest')
plt.figure('oldest')
ax = plt.axes(projection=projection)
ax.set_global()
ax.coastlines()
ax.gridlines()
for i in tqdm(range(0,n,1)):
    c = ax.pcolor(lon_points[-i,:,:],lat_points[-i,:,:],im_points[-i,:,:], transform=data_crs)
plt.colorbar(c,ax=ax)
#plt.pcolormesh(np.reshape(lon_points[0:100,:,:],(a,b*100)), np.reshape(lat_points[0:100,:,:],(a,b*100)), np.reshape(im_points[0:100,:,:],(a,b*100)))

plt.show()   

In [7]:
from skyfield.framelib import itrs
from skyfield.positionlib import Geocentric, ICRF
from skyfield.units import Distance
from numpy.linalg import norm
from scipy.optimize import minimize_scalar
from skyfield.api import wgs84 
from skyfield.api import load
from skyfield.units import Distance
from scipy.spatial.transform import Rotation as R
from scipy.interpolate import CubicSpline
from mats_l1_processing.pointing import pix_deg
import numpy as np
import datetime as DT
from datetime import datetime, timedelta, timezone
from pyarrow import fs
import boto3
import pyarrow as pa  # type: ignore
import pyarrow.dataset as ds  # type: ignore
from pandas import DataFrame, Timestamp  # type: ignore
from skyfield import api as sfapi
from scipy.interpolate import RegularGridInterpolator
import math
from mats_utils.geolocation.coordinates import deg_map,findsurface

def NADIR_geolocation(ccditem,x_sample=None,y_sample=None,interp_method='cubic'):
    """
    Function to get the latitude, longitude and solar zenith angle map for each pixel of the image.
    The values are calculated for some points and then interpolated for each pixel.
    WARNING : no images are flipped
    
    Parameters
    ----------
    ccditem : CCDitem
        measurement
    x_sample : int
        number of geolocated points along the x axis used for the interpolation. Default value is None, which means that there is no interpolation along the x-axis (each value is computed)
    y_step : int
        number of geolocated points along the y axis used for the interpolation. Default value is None, which means that there is no interpolation along the y-axis (each value is computed)
    interp_method :
        interpolation method : 'linear', 'nearest', 'slinear', 'cubic', 'quintic' and 'pchip'
        WARNING : choose the minimum x and y sampling according to the interpolation method
            
    Returns
    -------
    lat_map : array[float]
        map giving the latitude for each pixel in the image
    lon_map : array[float]
        map giving the longitude for each pixel in the image
    sza_map : array[float]
        map giving the solar zenith angle for each pixel in the image
    """
    im = ccditem['IMAGE']
    x_deg_map, y_deg_map = deg_map(ccditem) # creating angle deviation map for each pixel (degrees)
    a,b = np.shape(im)

    metoOHB  = R.from_matrix([[0,0,-1],[0,-1,0],[-1,0,0]])
    ts=sfapi.load.timescale()
    t=ts.from_datetime((ccditem['EXPDate']+timedelta(seconds=ccditem['TEXPMS']/(2*1000))).replace(tzinfo=sfapi.utc)) # exposure time (middle of the exposure timespan)  
    q=ccditem.afsAttitudeState
    quat=R.from_quat(np.roll(q,-1)) # quaternion of MATS attitude (for the satellite frame) 
    pos=ccditem.afsGnssStateJ2000[0:3] # position of MATS
    
    if x_sample == None:
        x_sample = b
    if y_sample == None:
        y_sample = a
    
    interpolation = True
    if x_sample == a and y_sample == b:
        interpolation = False

    x_deg_sample = np.zeros((a,b))
    y_deg_sample = np.zeros((a,b))

    xd = np.linspace(np.min(x_deg_map),np.max(x_deg_map),x_sample) # sampled angles on the x axis
    yd = np.linspace(np.min(y_deg_map),np.max(y_deg_map),y_sample) # sampled angles on the y axis
    x_deg_sample,y_deg_sample = np.meshgrid(xd,yd)

    if not interpolation:
        y_deg_sample,x_deg_sample = y_deg_map,x_deg_map

    LAT = np.zeros((y_sample,x_sample))
    LON = np.zeros((y_sample,x_sample))
    SZA = np.zeros((y_sample,x_sample))

    # computing the latitude, longitude and solar zenith angles at the intersection of the line of sight and the earth surface
    # only the line of sights from some sampled pixels are computed
    for i in range(y_sample):
        for j in range(x_sample):
                # angular transformations
                # rotation from the line of sight of the LIMB imager to the line of sight of the NADIR pixel
                angle = R.from_euler('XYZ', [x_deg_sample[i,j],-(90-23)+y_deg_sample[i,j],0] , degrees=True).apply([1, 0, 0])
                FOV = quat.apply(metoOHB.apply(angle)) # attitude state for the line of sight of the NADIR pixel    
                # finding the distance between the point pos and the Geoid along the line of sight
                res = findsurface(t,pos,FOV)
                newp = pos + res.x * FOV 
                newp = ICRF(Distance(m=newp).au, t=t, center=399) # point at the intersection between the line of sight at the pixel and the Geoid surface
                LAT[i,j]=wgs84.subpoint(newp).latitude.degrees # latitude of the point
                LON[i,j]=wgs84.subpoint(newp).longitude.degrees # longitude of the point E [-180,+180] 

                # finding the solar zenith angle of the point
                planets = sfapi.load('de421.bsp')
                earth=planets['Earth']
                sun=planets['Sun']
                SZA[i,j]=90-((earth+wgs84.subpoint(newp)).at(t).observe(sun).apparent().altaz())[0].degrees
    
    # to get a continuous longitudinal field
    if np.max(LON)-np.min(LON) > 300: # this condition is met if points are on both sides of the -180/+180 deg line
        LON = np.where(LON<0,LON+360,LON)

    if interpolation:
        # interpolating the results along all the pixels
        interp_lat = RegularGridInterpolator((yd,xd),LAT,interp_method,bounds_error=False,fill_value=None) 
        interp_lon = RegularGridInterpolator((yd,xd),LON,interp_method,bounds_error=False,fill_value=None)
        interp_sza = RegularGridInterpolator((yd,xd),SZA,interp_method,bounds_error=False,fill_value=None)

        lat_map = interp_lat((y_deg_map,x_deg_map))
        lon_map = interp_lon((y_deg_map,x_deg_map))
        sza_map = interp_sza((y_deg_map,x_deg_map))
    else:
        
        lat_map = LAT
        lon_map = LON
        sza_map = SZA

    return(lat_map,lon_map,sza_map)